![Neptune + PyTorch Lightning](https://neptune.ai/wp-content/uploads/2023/09/lightning.svg)

# Neptune + PyTorch Lightning

<a target="_blank" href="https://colab.research.google.com/github/neptune-ai/examples/blob/main/integrations-and-supported-tools/pytorch-lightning/notebooks/Neptune_PyTorch_Lightning.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a><a target="_blank" href="https://github.com/neptune-ai/examples/blob/main/integrations-and-supported-tools/pytorch-lightning/notebooks/Neptune_PyTorch_Lightning.ipynb">
  <img alt="Open in GitHub" src="https://img.shields.io/badge/Open_in_GitHub-blue?logo=github&labelColor=black">
</a><a target="_blank" href="https://app.neptune.ai/o/common/org/pytorch-lightning-integration/runs/details?viewId=standard-view&detailsTab=dashboard&dashboardId=ab76dda6-4f0d-42cb-9676-e6c68acb806c&shortId=PTL-3159"> 
  <img alt="Explore in Neptune" src="https://neptune.ai/wp-content/uploads/2024/01/neptune-badge.svg">
</a><a target="_blank" href="https://docs.neptune.ai/integrations/lightning/">
  <img alt="View tutorial in docs" src="https://neptune.ai/wp-content/uploads/2024/01/docs-badge-2.svg">
</a>

## Introduction

This guide will show you how to:

* Create a `NeptuneLogger()` instance
* Use the logger to log metadata to Neptune

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

If you want to see the example logged to your own workspace instead:

  1. Create a Neptune account. [Register &rarr;](https://neptune.ai/register)
  1. Create a Neptune project that you will use for tracking metadata. For instructions, see [Creating a project](https://docs.neptune.ai/setup/creating_project) in the Neptune docs.

## Install Neptune and dependencies

In [ ]:
%pip install -U neptune lightning scikit-plot torchvision "scipy<1.12"
%pip install -U --user pydantic

**Note**: If running on Google Colab, restart the kernel and continue execution from the next cell to avoid a `ContextualVersionConflict` error.

This error is caused by Colab coming with `future==0.16.0` preinstalled, while `torchvision` updates this to a newer version.

## Import libraries

In [ ]:
import os

import numpy as np
import torch
from lightning import LightningModule, Trainer
from lightning.pytorch.loggers.neptune import NeptuneLogger
from sklearn.metrics import accuracy_score
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST

## Define hyperparameters

In [ ]:
params = {
    "batch_size": 16,
    "lr": 0.0001,
    "max_epochs": 2,
}

## Define LightningModule and DataLoader()

In [ ]:
# (neptune) define LightningModule with logging (self.log)
class MNISTModel(LightningModule):
    def __init__(self):
        super().__init__()
        self.training_step_outputs = []
        self.l1 = torch.nn.Linear(28 * 28, 10)

    def forward(self, x):
        return torch.relu(self.l1(x.view(x.size(0), -1)))

    def training_step(self, batch):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log("train/batch/loss", loss, prog_bar=False)

        y_true = y.cpu().detach().numpy()
        y_pred = y_hat.argmax(axis=1).cpu().detach().numpy()
        acc = accuracy_score(y_true, y_pred)
        self.log("train/batch/acc", acc)

        self.training_step_outputs.append({"loss": loss, "y_true": y_true, "y_pred": y_pred})

        return {"loss": loss, "y_true": y_true, "y_pred": y_pred}

    def on_train_epoch_end(self):
        loss = np.array([])
        y_true = np.array([])
        y_pred = np.array([])
        for results_dict in self.training_step_outputs:
            loss = np.append(loss, results_dict["loss"].detach().numpy())
            y_true = np.append(y_true, results_dict["y_true"])
            y_pred = np.append(y_pred, results_dict["y_pred"])
        acc = accuracy_score(y_true, y_pred)
        self.log("train/epoch/loss", loss.mean())
        self.log("train/epoch/acc", acc)
        self.training_step_outputs.clear()  # free memory

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=params["lr"])


# init model
mnist_model = MNISTModel()

# init DataLoader from MNIST dataset
train_ds = MNIST(os.getcwd(), train=True, download=True, transform=transforms.ToTensor())
train_loader = DataLoader(train_ds, batch_size=params["batch_size"])

## Create a NeptuneLogger() instance

To create a new run for tracking the metadata, you tell Neptune who you are (`api_token`) and where to send the data (`project`).

You can use the default code cell below to create an anonymous run in a public project. **Note**: Public projects are cleaned regularly, so anonymous runs are only stored temporarily.

### Log to your own project instead

Replace the code below with the following:

```python
from getpass import getpass

neptune_logger = NeptuneLogger(
    project="workspace-name/project-name",  # replace with your own (see instructions below)
    api_token=getpass("Enter your Neptune API token: "),
    tags=["simple", "notebook"],
    log_model_checkpoints=False,  # Update to True to log model checkpoints
)
```

To find your API token and full project name:

1. [Log in to Neptune](https://app.neptune.ai/).
1. In the bottom-left corner, expand your user menu and select **Get your API token**.
1. You can copy the project path from the project settings (select **Edit project details**).

For more help, see [Setting Neptune credentials](https://docs.neptune.ai/setup/setting_credentials) in the Neptune docs.

In [ ]:
from neptune import ANONYMOUS_API_TOKEN

neptune_logger = NeptuneLogger(
    api_key=ANONYMOUS_API_TOKEN,
    project="common/pytorch-lightning-integration",
    tags=["simple", "notebook"],
    log_model_checkpoints=False,  # Update to True to log model checkpoints
)

## Initialize a trainer and pass neptune_logger

In [ ]:
# (neptune) initialize a trainer and pass neptune_logger
trainer = Trainer(
    logger=neptune_logger,
    max_epochs=params["max_epochs"],
)

## Log hyperparameters to the run

In [ ]:
neptune_logger.log_hyperparams(params=params)

## Train the model and log metadata to the run

In [ ]:
trainer.fit(mnist_model, train_loader)

## Stop logging

<font color=red>**Warning:**</font><br>
Once you are done logging, you should stop the run using the `stop()` method, which is a method of the run object. The run object itself is part of the `neptune_logger`.
This is needed only while logging from a notebook environment. While logging through a script, Neptune automatically stops tracking once the script has completed execution.

In [ ]:
neptune_logger.experiment.stop()

## Analyze logged metadata in the Neptune app

To explore the metadata in Neptune, follow the link in the console output.

It looks something like this: https://app.neptune.ai/o/common/org/pytorch-lightning-integration/e/PTL-3159